<a href="https://colab.research.google.com/github/ciepielajan/GeoFabrik_Data_Extract/blob/main/GEOfabrik_analiza_zawartosci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import folium
import pandas as pd
from plotnine import *
!pip -q install geopandas
from geopandas import GeoDataFrame

%matplotlib inline

     |████████████████████████████████| 1.0MB 17.6MB/s 
     |████████████████████████████████| 14.8MB 251kB/s 
     |████████████████████████████████| 6.5MB 31.8MB/s 


Load data from shapefile downloaded from http://download.geofabrik.de/

In [2]:
!wget "http://download.geofabrik.de/europe/poland/malopolskie-latest-free.shp.zip"  -O map.shp.zip

--2021-04-05 07:09:00--  http://download.geofabrik.de/europe/poland/malopolskie-latest-free.shp.zip
Resolving download.geofabrik.de (download.geofabrik.de)... 116.202.112.212, 138.201.219.183, 88.99.142.44
Connecting to download.geofabrik.de (download.geofabrik.de)|116.202.112.212|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232252554 (221M) [application/zip]
Saving to: ‘map.shp.zip’

map.shp.zip         100%[===================>] 221.49M  27.8MB/s    in 8.3s    

2021-04-05 07:09:09 (26.6 MB/s) - ‘map.shp.zip’ saved [232252554/232252554]



In [3]:
!unzip -q map.shp.zip -d /content/maps

In [4]:
from os import listdir

files_dir =  listdir("/content/maps")
files_list = []
# files_name_list = []
for names in files_dir:
    if names.endswith(".shp"):
        files_list.append(names)
        # files_name_list.append(names.split("osm_")[1].split("_free")[0])

files_list = sorted(files_list)
# files_name_list = sorted(files_name_list)
print(pd.Series(files_list))

ilość plików 18
gis_osm_buildings_a_free_1.shp
gis_osm_landuse_a_free_1.shp
gis_osm_natural_a_free_1.shp
gis_osm_natural_free_1.shp
gis_osm_places_a_free_1.shp
gis_osm_places_free_1.shp
gis_osm_pofw_a_free_1.shp
gis_osm_pofw_free_1.shp
gis_osm_pois_a_free_1.shp
gis_osm_pois_free_1.shp
gis_osm_railways_free_1.shp
gis_osm_roads_free_1.shp
gis_osm_traffic_a_free_1.shp
gis_osm_traffic_free_1.shp
gis_osm_transport_a_free_1.shp
gis_osm_transport_free_1.shp
gis_osm_water_a_free_1.shp
gis_osm_waterways_free_1.shp


# hurtowe załadwanie wszytkich plików

In [ ]:
data_frames = {}
for name in files_list:
  name_df = name.split(".")[0]
  data_frames[name_df] = GeoDataFrame.from_file(f'maps/{name}', encoding='utf-8')

In [ ]:
data_frames.keys()

dict_keys(['gis_osm_buildings_a_free_1', 'gis_osm_landuse_a_free_1', 'gis_osm_natural_a_free_1', 'gis_osm_natural_free_1', 'gis_osm_places_a_free_1', 'gis_osm_places_free_1', 'gis_osm_pofw_a_free_1', 'gis_osm_pofw_free_1', 'gis_osm_pois_a_free_1', 'gis_osm_pois_free_1', 'gis_osm_railways_free_1', 'gis_osm_roads_free_1', 'gis_osm_traffic_a_free_1', 'gis_osm_traffic_free_1', 'gis_osm_transport_a_free_1', 'gis_osm_transport_free_1', 'gis_osm_water_a_free_1', 'gis_osm_waterways_free_1'])

In [ ]:
data_frames.get("gis_osm_waterways_free_1")

# translate label

In [6]:
!pip -q install textblob
from textblob import TextBlob
import textblob.exceptions # obsługa błędu jeżli brak tłumaczenia

In [5]:
fclass_lists = []
for name in files_list:
  fclass_frame = GeoDataFrame.from_file(f'maps/{name}', encoding='utf-8').fclass.value_counts().index
  for x in fclass_frame:
     fclass_lists.append(x)

In [12]:
def get_translate(word, from_lang, to_lang):
  try:
    translate = TextBlob(word).translate(from_lang=from_lang, to=to_lang)
  except textblob.exceptions.NotTranslated:
    translate = word

  return str(translate)

In [15]:
fclass_eng_to_pl = {}
for word in fclass_lists:
  fclass_eng_to_pl[word] = get_translate(word, "en", "pl")

#buildings_a

In [117]:
%time
buildings_a = GeoDataFrame.from_file('maps/gis_osm_buildings_a_free_1.shp', encoding='utf-8')
buildings_a["fclass_PL"] = buildings_a["fclass"].replace(fclass_eng_to_pl)
print(buildings_a.shape)
buildings_a.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 19.8 µs
(1349204, 7)


,osm_id,code,fclass,name,type,geometry,fclass_PL
0,19796313,1500,building,Hilton Garden Inn Krakow Airport,hotel,"POLYGON ((19.80211 50.07165, 19.80322 50.07180...",budynek
1,23062762,1500,building,Kościół pw. Świętego Mikołaja Biskupa,church,"POLYGON ((20.42797 49.96935, 20.42801 49.96936...",budynek
2,23068834,1500,building,Dzwonnica drewniana XV w.,None,"POLYGON ((20.42812 49.96904, 20.42823 49.96908...",budynek
3,23256528,1500,building,Sukiennice,public,"POLYGON ((19.93679 50.06135, 19.93681 50.06137...",budynek
4,24208843,1500,building,"Główna łaźnia ""Sauna""",None,"POLYGON ((19.16641 50.03883, 19.16643 50.03947...",budynek


In [118]:
label_counts = buildings_a[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,building,budynek,1349204


In [119]:
label_counts.to_markdown()

'|    | fclass   | fclass_PL   |   value counts |\n|---:|:---------|:------------|---------------:|\n|  0 | building | budynek     |        1349204 |'

In [130]:
label_counts = buildings_a[["type"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts[:15]

,type,value counts
0,house,44552
1,residential,15172
2,detached,12418
3,farm_auxiliary,7761
4,apartments,6975
5,garage,6775
6,greenhouse,2973
7,shed,2106
8,industrial,2024
9,garages,1710


In [42]:
label_counts.to_markdown()

'|     | type                 |   value counts |\n|----:|:---------------------|---------------:|\n|   0 | house                |          44552 |\n|   1 | residential          |          15172 |\n|   2 | detached             |          12418 |\n|   3 | farm_auxiliary       |           7761 |\n|   4 | apartments           |           6975 |\n|   5 | garage               |           6775 |\n|   6 | greenhouse           |           2973 |\n|   7 | shed                 |           2106 |\n|   8 | industrial           |           2024 |\n|   9 | garages              |           1710 |\n|  10 | retail               |           1617 |\n|  11 | terrace              |           1415 |\n|  12 | service              |           1391 |\n|  13 | school               |           1324 |\n|  14 | church               |           1304 |\n|  15 | roof                 |           1299 |\n|  16 | commercial           |           1103 |\n|  17 | barn                 |            982 |\n|  18 | constructio

#landuse_a

In [43]:
%time
landuse_a = GeoDataFrame.from_file('maps/gis_osm_landuse_a_free_1.shp', encoding='utf-8')
landuse_a["fclass_PL"] = landuse_a["fclass"].replace(fclass_eng_to_pl)
print(landuse_a.shape)
landuse_a.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs
(182239, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,20190259,7201,forest,None,"POLYGON ((19.80347 50.10261, 19.80353 50.10273...",las
1,20190402,7201,forest,None,"POLYGON ((19.80738 50.09817, 19.80747 50.09828...",las
2,20190511,7201,forest,None,"POLYGON ((19.82236 50.09557, 19.82337 50.09550...",las
3,20193458,7218,grass,None,"POLYGON ((20.05270 50.07950, 20.05278 50.07960...",trawa
4,22938927,7201,forest,None,"POLYGON ((19.84609 50.01414, 19.84612 50.01432...",las


In [44]:
label_counts = landuse_a[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,forest,las,49400
1,residential,osiedle mieszkaniowe,32980
2,farmland,pole uprawne,27601
3,scrub,zarośla,21638
4,meadow,łąka,17930
5,grass,trawa,16450
6,orchard,sad owocowy,7339
7,farmyard,obejście,2344
8,industrial,przemysłowy,1818
9,cemetery,cmentarz,1372


In [45]:
label_counts.to_markdown()

'|    | fclass            | fclass_PL            |   value counts |\n|---:|:------------------|:---------------------|---------------:|\n|  0 | forest            | las                  |          49400 |\n|  1 | residential       | osiedle mieszkaniowe |          32980 |\n|  2 | farmland          | pole uprawne         |          27601 |\n|  3 | scrub             | zarośla              |          21638 |\n|  4 | meadow            | łąka                 |          17930 |\n|  5 | grass             | trawa                |          16450 |\n|  6 | orchard           | sad owocowy          |           7339 |\n|  7 | farmyard          | obejście             |           2344 |\n|  8 | industrial        | przemysłowy          |           1818 |\n|  9 | cemetery          | cmentarz             |           1372 |\n| 10 | retail            | sprzedaż             |            900 |\n| 11 | commercial        | Reklama w telewizji  |            686 |\n| 12 | park              | park                

#natural_a

In [46]:
%time
natural_a = GeoDataFrame.from_file('maps/gis_osm_natural_a_free_1.shp', encoding='utf-8')
natural_a["fclass_PL"] = natural_a["fclass"].replace(fclass_eng_to_pl)
print(natural_a.shape)
natural_a.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.82 µs
(110, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,25156249,4141,beach,kąpielisko strzeżone,"POLYGON ((19.79176 50.04785, 19.79182 50.04787...",plaża
1,25156251,4141,beach,None,"POLYGON ((19.79706 50.04863, 19.79706 50.04875...",plaża
2,25157073,4141,beach,Kryspinów Plaża Naturystyczna,"POLYGON ((19.78679 50.05572, 19.78715 50.05571...",plaża
3,25450792,4141,beach,None,"POLYGON ((19.44993 50.13863, 19.44999 50.13883...",plaża
4,25450793,4141,beach,None,"POLYGON ((19.45208 50.13762, 19.45223 50.13770...",plaża


In [47]:
label_counts = natural_a[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,beach,plaża,75
1,cliff,Klif,21
2,glacier,lodowiec,12
3,spring,wiosna,2


In [48]:
label_counts.to_markdown()

'|    | fclass   | fclass_PL   |   value counts |\n|---:|:---------|:------------|---------------:|\n|  0 | beach    | plaża       |             75 |\n|  1 | cliff    | Klif        |             21 |\n|  2 | glacier  | lodowiec    |             12 |\n|  3 | spring   | wiosna      |              2 |'

#natural

In [49]:
%time
natural = GeoDataFrame.from_file('maps/gis_osm_natural_free_1.shp', encoding='utf-8')
natural["fclass_PL"] = natural["fclass"].replace(fclass_eng_to_pl)
print(natural.shape)
natural.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.3 µs
(51160, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,26864278,4111,peak,Pilsko,POINT (19.31675 49.52800),szczyt
1,26864983,4111,peak,Turbacz,POINT (20.11138 49.54282),szczyt
2,31943589,4111,peak,Dziamera,POINT (21.30261 49.55284),szczyt
3,226153423,4111,peak,Kopiec Kościuszki,POINT (19.89335 50.05492),szczyt
4,273454953,4121,tree,None,POINT (19.91568 50.06385),drzewo


In [50]:
label_counts = natural[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,tree,drzewo,48164
1,peak,szczyt,2324
2,cave_entrance,cave_entrance,415
3,spring,wiosna,238
4,cliff,Klif,18
5,beach,plaża,1


In [51]:
label_counts.to_markdown()

'|    | fclass        | fclass_PL     |   value counts |\n|---:|:--------------|:--------------|---------------:|\n|  0 | tree          | drzewo        |          48164 |\n|  1 | peak          | szczyt        |           2324 |\n|  2 | cave_entrance | cave_entrance |            415 |\n|  3 | spring        | wiosna        |            238 |\n|  4 | cliff         | Klif          |             18 |\n|  5 | beach         | plaża         |              1 |'

#places_a

In [52]:
%time
places_a = GeoDataFrame.from_file('maps/gis_osm_places_a_free_1.shp', encoding='utf-8')
places_a["fclass_PL"] = places_a["fclass"].replace(fclass_eng_to_pl)
print(places_a.shape)
places_a.head()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.54 µs
(34, 7)


,osm_id,code,fclass,population,name,geometry,fclass_PL
0,69553535,1050,locality,0,Hala Krupowa,"POLYGON ((19.65468 49.62424, 19.65506 49.62446...",miejscowość
1,130583808,1020,island,0,None,"POLYGON ((20.07783 50.25781, 20.07798 50.25802...",wyspa
2,168403564,1020,island,0,Ptasia Wyspa,"POLYGON ((20.24106 49.45946, 20.24108 49.45953...",wyspa
3,3288260,1003,village,0,Książniczki,"POLYGON ((20.00059 50.13281, 20.00079 50.13293...",wioska
4,259975428,1050,locality,0,Czoło,"POLYGON ((20.23166 49.52170, 20.23166 49.52193...",miejscowość


In [53]:
label_counts = places_a[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,island,wyspa,15
1,locality,miejscowość,9
2,suburb,przedmieście,8
3,village,wioska,1
4,farm,gospodarstwo rolne,1


In [54]:
label_counts.to_markdown()

'|    | fclass   | fclass_PL          |   value counts |\n|---:|:---------|:-------------------|---------------:|\n|  0 | island   | wyspa              |             15 |\n|  1 | locality | miejscowość        |              9 |\n|  2 | suburb   | przedmieście       |              8 |\n|  3 | village  | wioska             |              1 |\n|  4 | farm     | gospodarstwo rolne |              1 |'

#places

In [55]:
%time
places = GeoDataFrame.from_file('maps/gis_osm_places_free_1.shp', encoding='utf-8')
places["fclass_PL"] = places["fclass"].replace(fclass_eng_to_pl)
print(places.shape)
places.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs
(18002, 7)


,osm_id,code,fclass,population,name,geometry,fclass_PL
0,26035799,1003,village,364,Osturňa,POINT (20.23333 49.33333),wioska
1,26036002,1004,hamlet,139,Legnava,POINT (20.84981 49.33675),mała wioska
2,26036012,1003,village,446,Malý Lipník,POINT (20.79956 49.33732),wioska
3,26036092,1003,village,361,Veľká Franková,POINT (20.29345 49.33804),wioska
4,26036193,1003,village,1319,Suchá Hora,POINT (19.79011 49.36631),wioska


In [56]:
label_counts = places[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,hamlet,mała wioska,14849
1,village,wioska,1976
2,locality,miejscowość,1065
3,town,miasto,62
4,suburb,przedmieście,45
5,city,Miasto,3
6,island,wyspa,1
7,farm,gospodarstwo rolne,1


In [57]:
label_counts.to_markdown()

'|    | fclass   | fclass_PL          |   value counts |\n|---:|:---------|:-------------------|---------------:|\n|  0 | hamlet   | mała wioska        |          14849 |\n|  1 | village  | wioska             |           1976 |\n|  2 | locality | miejscowość        |           1065 |\n|  3 | town     | miasto             |             62 |\n|  4 | suburb   | przedmieście       |             45 |\n|  5 | city     | Miasto             |              3 |\n|  6 | island   | wyspa              |              1 |\n|  7 | farm     | gospodarstwo rolne |              1 |'

#pofw_a

In [58]:
%time
pofw_a = GeoDataFrame.from_file('maps/gis_osm_pofw_a_free_1.shp', encoding='utf-8')
pofw_a["fclass_PL"] = pofw_a["fclass"].replace(fclass_eng_to_pl)
print(pofw_a.shape)
pofw_a.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 10 µs
(3214, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,23062762,3100,christian,Kościół pw. Świętego Mikołaja Biskupa,"POLYGON ((20.42797 49.96935, 20.42801 49.96936...",chrześcijanin
1,23257702,3100,christian,Cmentarz parafialny Salwator,"POLYGON ((19.90225 50.05298, 19.90231 50.05298...",chrześcijanin
2,24264572,3100,christian,Cmentarz parafialny,"POLYGON ((21.18862 49.63513, 21.18891 49.63523...",chrześcijanin
3,25487604,3100,christian,Kaplica pw. Chrystusa Odkupiciela w Krakowie,"POLYGON ((19.98309 50.10132, 19.98329 50.10136...",chrześcijanin
4,26195267,3100,christian,Kościół pw. Wniebowzięcia Najświętszej Marii P...,"POLYGON ((19.93883 50.06164, 19.93884 50.06165...",chrześcijanin


In [59]:
label_counts = pofw_a[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,christian,chrześcijanin,3027
1,christian_catholic,Chrześcijański katolik,80
2,jewish,żydowski,66
3,christian_orthodox,christian_orthodox,29
4,christian_lutheran,christian_lutheran,6
5,christian_protestant,christian_protestant,3
6,christian_evangelical,christian_evangelical,2
7,hindu,Hindus,1


In [60]:
label_counts.to_markdown()

'|    | fclass                | fclass_PL              |   value counts |\n|---:|:----------------------|:-----------------------|---------------:|\n|  0 | christian             | chrześcijanin          |           3027 |\n|  1 | christian_catholic    | Chrześcijański katolik |             80 |\n|  2 | jewish                | żydowski               |             66 |\n|  3 | christian_orthodox    | christian_orthodox     |             29 |\n|  4 | christian_lutheran    | christian_lutheran     |              6 |\n|  5 | christian_protestant  | christian_protestant   |              3 |\n|  6 | christian_evangelical | christian_evangelical  |              2 |\n|  7 | hindu                 | Hindus                 |              1 |'

#pofw

In [61]:
%time
pofw = GeoDataFrame.from_file('maps/gis_osm_pofw_free_1.shp', encoding='utf-8')
pofw["fclass_PL"] = pofw["fclass"].replace(fclass_eng_to_pl)
print(pofw.shape)
pofw.head()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs
(1718, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,672084028,3100,christian,None,POINT (20.71168 49.41291),chrześcijanin
1,672084039,3100,christian,None,POINT (20.72726 49.41456),chrześcijanin
2,672084043,3100,christian,None,POINT (20.72149 49.41717),chrześcijanin
3,672084046,3100,christian,None,POINT (20.69901 49.41636),chrześcijanin
4,672084110,3100,christian,None,POINT (20.79427 49.33344),chrześcijanin


In [62]:
label_counts = pofw[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,christian,chrześcijanin,1335
1,christian_catholic,Chrześcijański katolik,356
2,jewish,żydowski,10
3,christian_orthodox,christian_orthodox,10
4,christian_protestant,christian_protestant,4
5,muslim,muzułmański,1
6,christian_methodist,christian_methodist,1
7,christian_lutheran,christian_lutheran,1


In [63]:
label_counts.to_markdown()

'|    | fclass               | fclass_PL              |   value counts |\n|---:|:---------------------|:-----------------------|---------------:|\n|  0 | christian            | chrześcijanin          |           1335 |\n|  1 | christian_catholic   | Chrześcijański katolik |            356 |\n|  2 | jewish               | żydowski               |             10 |\n|  3 | christian_orthodox   | christian_orthodox     |             10 |\n|  4 | christian_protestant | christian_protestant   |              4 |\n|  5 | muslim               | muzułmański            |              1 |\n|  6 | christian_methodist  | christian_methodist    |              1 |\n|  7 | christian_lutheran   | christian_lutheran     |              1 |'

#pois_a

In [64]:
%time
pois_a = GeoDataFrame.from_file('maps/gis_osm_pois_a_free_1.shp', encoding='utf-8')
pois_a["fclass_PL"] = pois_a["fclass"].replace(fclass_eng_to_pl)
print(pois_a.shape)
pois_a.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.78 µs
(16099, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,19796313,2401,hotel,Hilton Garden Inn Krakow Airport,"POLYGON ((19.80211 50.07165, 19.80322 50.07180...",hotel
1,19803351,2252,pitch,None,"POLYGON ((20.05971 50.06508, 20.06070 50.06514...",smoła
2,19841872,2252,pitch,None,"POLYGON ((19.90818 50.06320, 19.90840 50.06376...",smoła
3,19842194,2258,track,None,"POLYGON ((19.91531 50.07965, 19.91535 50.07977...",tor
4,23062643,2204,park,None,"POLYGON ((20.42388 49.96833, 20.42399 49.96837...",park


In [65]:
label_counts = pois_a[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,pitch,smoła,3483
1,graveyard,cmentarz,1372
2,playground,plac zabaw,1322
3,school,szkoła,1082
4,shelter,schronienie,704
...,...,...,...
115,drinking_water,woda pitna,2
116,travel_agent,agent biura podróży,2
117,nightclub,Klub nocny,2
118,outdoor_shop,outdoor_shop,1


In [66]:
label_counts.to_markdown()

'|     | fclass            | fclass_PL                      |   value counts |\n|----:|:------------------|:-------------------------------|---------------:|\n|   0 | pitch             | smoła                          |           3483 |\n|   1 | graveyard         | cmentarz                       |           1372 |\n|   2 | playground        | plac zabaw                     |           1322 |\n|   3 | school            | szkoła                         |           1082 |\n|   4 | shelter           | schronienie                    |            704 |\n|   5 | park              | park                           |            644 |\n|   6 | fire_station      | stacja pożarowa                |            447 |\n|   7 | convenience       | wygoda                         |            441 |\n|   8 | supermarket       | supermarket                    |            403 |\n|   9 | ruins             | gruzy                          |            348 |\n|  10 | hotel             | hotel                  

#pois

In [67]:
%time
pois = GeoDataFrame.from_file('maps/gis_osm_pois_free_1.shp', encoding='utf-8')
pois["fclass_PL"] = pois["fclass"].replace(fclass_eng_to_pl)
print(pois.shape)
pois.head()

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.4 µs
(33844, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,25606107,2701,tourist_info,Rysy,POINT (20.08808 49.17946),informacja_turystyczna
1,26860250,2422,camp_site,Camp-Cup,POINT (19.62601 49.57885),obozowisko
2,26860273,2422,camp_site,Harenda,POINT (19.98500 49.32417),obozowisko
3,26860287,2422,camp_site,Krakowianka,POINT (19.92453 50.01552),obozowisko
4,31944307,2404,guesthouse,Wawrzka,POINT (20.98333 49.58333),domek gościnny


In [68]:
label_counts = pois[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,bench,ławka,5008
1,convenience,wygoda,2034
2,tourist_info,informacja_turystyczna,1726
3,wayside_shrine,wayside_shrine,1606
4,restaurant,restauracja,1481
...,...,...,...
127,track,tor,2
128,embassy,ambasada,1
129,prison,więzienie,1
130,water_mill,Młyn wodny,1


In [69]:
label_counts.to_markdown()

'|     | fclass              | fclass_PL                      |   value counts |\n|----:|:--------------------|:-------------------------------|---------------:|\n|   0 | bench               | ławka                          |           5008 |\n|   1 | convenience         | wygoda                         |           2034 |\n|   2 | tourist_info        | informacja_turystyczna         |           1726 |\n|   3 | wayside_shrine      | wayside_shrine                 |           1606 |\n|   4 | restaurant          | restauracja                    |           1481 |\n|   5 | atm                 | bankomat                       |           1016 |\n|   6 | waste_basket        | kosz_odpadowy                  |            976 |\n|   7 | pharmacy            | Apteka                         |            811 |\n|   8 | wayside_cross       | wayside_cross                  |            707 |\n|   9 | memorial            | memoriał                       |            696 |\n|  10 | fire_station       

#railways

In [73]:
%time
railways = GeoDataFrame.from_file('maps/gis_osm_railways_free_1.shp', encoding='utf-8')
railways["fclass_PL"] = railways["fclass"].replace(fclass_eng_to_pl)
print(railways.shape)
railways.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 10 µs
(6116, 9)


,osm_id,code,fclass,name,layer,bridge,tunnel,geometry,fclass_PL
0,4820685,6101,rail,None,0,F,F,"LINESTRING (19.81982 50.10854, 19.83200 50.099...",szyna
1,5901391,6101,rail,None,0,F,F,"LINESTRING (20.15380 50.02779, 20.15478 50.030...",szyna
2,5901548,6101,rail,None,0,F,F,"LINESTRING (19.96535 50.04225, 19.96580 50.041...",szyna
3,5901574,6101,rail,None,0,F,F,"LINESTRING (20.37067 49.98760, 20.38054 49.98492)",szyna
4,5901827,6101,rail,None,1,T,F,"LINESTRING (20.15380 50.02779, 20.15373 50.02761)",szyna


In [74]:
label_counts = railways[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,rail,szyna,5143
1,tram,tramwajowy,769
2,narrow_gauge,wąska szerokość,178
3,monorail,jednoszynowy,19
4,funicular,kolejka linowa,7


In [75]:
label_counts.to_markdown()

'|    | fclass       | fclass_PL       |   value counts |\n|---:|:-------------|:----------------|---------------:|\n|  0 | rail         | szyna           |           5143 |\n|  1 | tram         | tramwajowy      |            769 |\n|  2 | narrow_gauge | wąska szerokość |            178 |\n|  3 | monorail     | jednoszynowy    |             19 |\n|  4 | funicular    | kolejka linowa  |              7 |'

In [76]:
label_counts = railways["layer"].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,index,layer
0,0,5133
1,1,898
2,-1,43
3,2,40
4,-2,2


In [77]:
label_counts.to_markdown()

'|    |   index |   layer |\n|---:|--------:|--------:|\n|  0 |       0 |    5133 |\n|  1 |       1 |     898 |\n|  2 |      -1 |      43 |\n|  3 |       2 |      40 |\n|  4 |      -2 |       2 |'

In [78]:
label_counts = railways["bridge"].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,index,bridge
0,F,5267
1,T,849


In [79]:
label_counts.to_markdown()

'|    | index   |   bridge |\n|---:|:--------|---------:|\n|  0 | F       |     5267 |\n|  1 | T       |      849 |'

In [80]:
label_counts = railways["tunnel"].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,index,tunnel
0,F,6073
1,T,43


In [81]:
label_counts.to_markdown()

'|    | index   |   tunnel |\n|---:|:--------|---------:|\n|  0 | F       |     6073 |\n|  1 | T       |       43 |'

#roads

In [82]:
%time
roads = GeoDataFrame.from_file('maps/gis_osm_roads_free_1.shp', encoding='utf-8')
roads["fclass_PL"] = roads["fclass"].replace(fclass_eng_to_pl)
print(roads.shape)
roads.head()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.54 µs
(372824, 12)


,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry,fclass_PL
0,2954554,5122,residential,Ignacego Mościckiego,None,B,40,0,F,F,"LINESTRING (20.03436 50.08068, 20.03452 50.080...",osiedle mieszkaniowe
1,2954556,5115,tertiary,Stefana Żeromskiego,None,B,50,0,F,F,"LINESTRING (20.03605 50.07939, 20.03626 50.079...",trzeciorzędowy
2,2954557,5122,residential,None,None,B,30,0,F,F,"LINESTRING (20.03550 50.07982, 20.03566 50.079...",osiedle mieszkaniowe
3,2954558,5122,residential,None,None,B,30,0,F,F,"LINESTRING (20.03484 50.08193, 20.03516 50.081...",osiedle mieszkaniowe
4,3989492,5124,pedestrian,Floriańska,None,F,20,0,F,F,"LINESTRING (19.94134 50.06483, 19.94133 50.064...",pieszy


In [83]:
label_counts = roads[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,service,usługa,135933
1,footway,chodnik,59029
2,track,tor,53065
3,residential,osiedle mieszkaniowe,46687
4,path,ścieżka,15276
5,tertiary,trzeciorzędowy,12251
6,unclassified,niesklasyfikowane,7740
7,primary,podstawowa,5410
8,secondary,wtórny,5386
9,steps,kroki,5351


In [84]:
label_counts.to_markdown()

'|    | fclass         | fclass_PL            |   value counts |\n|---:|:---------------|:---------------------|---------------:|\n|  0 | service        | usługa               |         135933 |\n|  1 | footway        | chodnik              |          59029 |\n|  2 | track          | tor                  |          53065 |\n|  3 | residential    | osiedle mieszkaniowe |          46687 |\n|  4 | path           | ścieżka              |          15276 |\n|  5 | tertiary       | trzeciorzędowy       |          12251 |\n|  6 | unclassified   | niesklasyfikowane    |           7740 |\n|  7 | primary        | podstawowa           |           5410 |\n|  8 | secondary      | wtórny               |           5386 |\n|  9 | steps          | kroki                |           5351 |\n| 10 | track_grade5   | track_grade5         |           4269 |\n| 11 | cycleway       | ścieżka rowerowa     |           4023 |\n| 12 | track_grade1   | track_grade1         |           3947 |\n| 13 | track_grade4   | 

In [85]:
label_counts = roads["ref"].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,index,ref
0,28,778
1,94,668
2,7,636
3,79,537
4,75,411
...,...,...
164,P-36,1
165,2075k,1
166,44;781,1
167,3158,1


In [86]:
label_counts.to_markdown()

'|     | index      |   ref |\n|----:|:-----------|------:|\n|   0 | 28         |   778 |\n|   1 | 94         |   668 |\n|   2 | 7          |   636 |\n|   3 | 79         |   537 |\n|   4 | 75         |   411 |\n|   5 | A4         |   406 |\n|   6 | 776        |   296 |\n|   7 | 780        |   296 |\n|   8 | 44         |   287 |\n|   9 | 977        |   232 |\n|  10 | 964        |   216 |\n|  11 | 781        |   212 |\n|  12 | 47         |   197 |\n|  13 | 52         |   195 |\n|  14 | 969        |   187 |\n|  15 | 73         |   176 |\n|  16 | 975        |   165 |\n|  17 | S7         |   161 |\n|  18 | 933        |   159 |\n|  19 | 966        |   159 |\n|  20 | 965        |   156 |\n|  21 | 7;79       |   153 |\n|  22 | 791        |   132 |\n|  23 | 958        |   123 |\n|  24 | 794        |   104 |\n|  25 | 87         |   104 |\n|  26 | 768        |   101 |\n|  27 | 773        |    96 |\n|  28 | 49         |    96 |\n|  29 | 971        |    95 |\n|  30 | 957        |    93 |\n|  31 | 9

In [87]:
label_counts = roads["oneway"].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,index,oneway
0,B,358344
1,F,14463
2,T,17


In [88]:
label_counts.to_markdown()

'|    | index   |   oneway |\n|---:|:--------|---------:|\n|  0 | B       |   358344 |\n|  1 | F       |    14463 |\n|  2 | T       |       17 |'

In [89]:
label_counts = roads["maxspeed"].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,index,maxspeed
0,0,347930
1,50,11407
2,30,3988
3,40,2134
4,20,2017
5,90,1879
6,70,1844
7,140,553
8,60,414
9,100,315


In [90]:
label_counts.to_markdown()

'|    |   index |   maxspeed |\n|---:|--------:|-----------:|\n|  0 |       0 |     347930 |\n|  1 |      50 |      11407 |\n|  2 |      30 |       3988 |\n|  3 |      40 |       2134 |\n|  4 |      20 |       2017 |\n|  5 |      90 |       1879 |\n|  6 |      70 |       1844 |\n|  7 |     140 |        553 |\n|  8 |      60 |        414 |\n|  9 |     100 |        315 |\n| 10 |     120 |        128 |\n| 11 |      10 |         93 |\n| 12 |      80 |         53 |\n| 13 |       5 |         43 |\n| 14 |      15 |         18 |\n| 15 |      25 |          6 |\n| 16 |     110 |          1 |\n| 17 |     130 |          1 |'

In [91]:
label_counts = roads["layer"].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,index,layer
0,0,364203
1,1,7632
2,-1,774
3,-2,94
4,2,93
5,3,21
6,5,3
7,-5,2
8,-3,1
9,4,1


In [92]:
label_counts.to_markdown()

'|    |   index |   layer |\n|---:|--------:|--------:|\n|  0 |       0 |  364203 |\n|  1 |       1 |    7632 |\n|  2 |      -1 |     774 |\n|  3 |      -2 |      94 |\n|  4 |       2 |      93 |\n|  5 |       3 |      21 |\n|  6 |       5 |       3 |\n|  7 |      -5 |       2 |\n|  8 |      -3 |       1 |\n|  9 |       4 |       1 |'

In [93]:
label_counts = roads["bridge"].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,index,bridge
0,F,365303
1,T,7521


In [94]:
label_counts.to_markdown()

'|    | index   |   bridge |\n|---:|:--------|---------:|\n|  0 | F       |   365303 |\n|  1 | T       |     7521 |'

In [95]:
label_counts = roads["tunnel"].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,index,tunnel
0,F,370948
1,T,1876


In [96]:
label_counts.to_markdown()

'|    | index   |   tunnel |\n|---:|:--------|---------:|\n|  0 | F       |   370948 |\n|  1 | T       |     1876 |'

#traffic_a

In [97]:
%time
traffic_a = GeoDataFrame.from_file('maps/gis_osm_traffic_a_free_1.shp', encoding='utf-8')
traffic_a["fclass_PL"] = traffic_a["fclass"].replace(fclass_eng_to_pl)
print(traffic_a.shape)
traffic_a.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.3 µs
(11744, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,22787855,5260,parking,None,"POLYGON ((19.65908 49.25882, 19.65908 49.25910...",parking
1,23068881,5260,parking,None,"POLYGON ((20.43310 49.97005, 20.43326 49.97006...",parking
2,25042802,5332,weir,Stopień Wodny Przewóz,"POLYGON ((20.09762 50.04161, 20.09763 50.04163...",jaz
3,25091288,5260,parking,None,"POLYGON ((21.16171 49.65080, 21.16187 49.65093...",parking
4,25123245,5260,parking,None,"POLYGON ((19.93655 50.06769, 19.93739 50.06780...",parking


In [98]:
label_counts = traffic_a[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,parking,parking,11211
1,fuel,paliwo,254
2,parking_bicycle,parking_bicycle,109
3,pier,molo,67
4,dam,zapora,46
5,parking_multistorey,parking_multistorey,22
6,service,usługa,11
7,marina,przystań,11
8,parking_underground,parking_underground,10
9,slipway,pochylnia,2


In [99]:
label_counts.to_markdown()

'|    | fclass              | fclass_PL           |   value counts |\n|---:|:--------------------|:--------------------|---------------:|\n|  0 | parking             | parking             |          11211 |\n|  1 | fuel                | paliwo              |            254 |\n|  2 | parking_bicycle     | parking_bicycle     |            109 |\n|  3 | pier                | molo                |             67 |\n|  4 | dam                 | zapora              |             46 |\n|  5 | parking_multistorey | parking_multistorey |             22 |\n|  6 | service             | usługa              |             11 |\n|  7 | marina              | przystań            |             11 |\n|  8 | parking_underground | parking_underground |             10 |\n|  9 | slipway             | pochylnia           |              2 |\n| 10 | weir                | jaz                 |              1 |'

#traffic

In [100]:
%time
traffic = GeoDataFrame.from_file('maps/gis_osm_traffic_free_1.shp', encoding='utf-8')
traffic["fclass_PL"] = traffic["fclass"].replace(fclass_eng_to_pl)
print(traffic.shape)
traffic.head()

CPU times: user 15 µs, sys: 1e+03 ns, total: 16 µs
Wall time: 20.3 µs
(30026, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,29782991,5201,traffic_signals,None,POINT (20.03122 50.07678),traffic_signals
1,29782993,5204,crossing,None,POINT (20.02799 50.07927),przejście
2,29782996,5201,traffic_signals,None,POINT (20.02795 50.07986),traffic_signals
3,30375883,5204,crossing,None,POINT (19.94247 50.06449),przejście
4,30376964,5201,traffic_signals,None,POINT (19.90412 50.04919),traffic_signals


In [101]:
label_counts = traffic[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,crossing,przejście,16711
1,street_lamp,lampa uliczna,6422
2,parking_bicycle,parking_bicycle,2294
3,parking,parking,1372
4,traffic_signals,traffic_signals,1289
5,stop,zatrzymać,544
6,turning_circle,Obracające się koło,436
7,fuel,paliwo,391
8,weir,jaz,280
9,motorway_junction,motorway_junction,63


In [102]:
label_counts.to_markdown()

'|    | fclass              | fclass_PL           |   value counts |\n|---:|:--------------------|:--------------------|---------------:|\n|  0 | crossing            | przejście           |          16711 |\n|  1 | street_lamp         | lampa uliczna       |           6422 |\n|  2 | parking_bicycle     | parking_bicycle     |           2294 |\n|  3 | parking             | parking             |           1372 |\n|  4 | traffic_signals     | traffic_signals     |           1289 |\n|  5 | stop                | zatrzymać           |            544 |\n|  6 | turning_circle      | Obracające się koło |            436 |\n|  7 | fuel                | paliwo              |            391 |\n|  8 | weir                | jaz                 |            280 |\n|  9 | motorway_junction   | motorway_junction   |             63 |\n| 10 | waterfall           | wodospad            |             52 |\n| 11 | speed_camera        | fotoradar           |             43 |\n| 12 | parking_underground | park

#transport_a

In [103]:
%time
transport_a = GeoDataFrame.from_file('maps/gis_osm_transport_a_free_1.shp', encoding='utf-8')
transport_a["fclass_PL"] = transport_a["fclass"].replace(fclass_eng_to_pl)
print(transport_a.shape)
transport_a.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 9.06 µs
(638, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,19796272,5656,apron,None,"POLYGON ((19.78131 50.07318, 19.78147 50.07336...",Fartuch
1,26424475,5656,apron,None,"POLYGON ((20.19613 50.08498, 20.19929 50.08502...",Fartuch
2,39272551,5622,bus_station,Azory,"POLYGON ((19.90876 50.08659, 19.90877 50.08663...",Dworzec autobusowy
3,43457434,5622,bus_station,Dworzec Autobusowy,"POLYGON ((20.69748 49.61713, 20.69845 49.61749...",Dworzec autobusowy
4,49156766,5621,bus_stop,Prokocim Szpital,"POLYGON ((20.00093 50.01492, 20.00142 50.01518...",przystanek autobusowy


In [104]:
label_counts = transport_a[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,bus_stop,przystanek autobusowy,523
1,helipad,lądowisko dla helikopterów,42
2,bus_station,Dworzec autobusowy,35
3,taxi,Taxi,16
4,apron,Fartuch,9
5,airport,Lotnisko,8
6,tram_stop,przystanek tramwajowy,2
7,railway_station,stacja kolejowa,2
8,airfield,lotnisko,1


In [105]:
label_counts.to_markdown()

'|    | fclass          | fclass_PL                  |   value counts |\n|---:|:----------------|:---------------------------|---------------:|\n|  0 | bus_stop        | przystanek autobusowy      |            523 |\n|  1 | helipad         | lądowisko dla helikopterów |             42 |\n|  2 | bus_station     | Dworzec autobusowy         |             35 |\n|  3 | taxi            | Taxi                       |             16 |\n|  4 | apron           | Fartuch                    |              9 |\n|  5 | airport         | Lotnisko                   |              8 |\n|  6 | tram_stop       | przystanek tramwajowy      |              2 |\n|  7 | railway_station | stacja kolejowa            |              2 |\n|  8 | airfield        | lotnisko                   |              1 |'

#transport

In [106]:
%time
transport = GeoDataFrame.from_file('maps/gis_osm_transport_free_1.shp', encoding='utf-8')
transport["fclass_PL"] = transport["fclass"].replace(fclass_eng_to_pl)
print(transport.shape)
transport.head()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.78 µs
(10107, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,41943557,5601,railway_station,Kraków Mydlniki,POINT (19.85970 50.08360),stacja kolejowa
1,48267476,5601,railway_station,Bochnia,POINT (20.43151 49.97729),stacja kolejowa
2,48522023,5601,railway_station,Kraków Batowice,POINT (19.99547 50.10753),stacja kolejowa
3,48523754,5601,railway_station,Kraków Nowa Huta,POINT (20.12950 50.09058),stacja kolejowa
4,48526714,5602,railway_halt,Słomniki Miasto,POINT (20.07637 50.23587),Railway_halt


In [107]:
label_counts = transport[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,bus_stop,przystanek autobusowy,9391
1,tram_stop,przystanek tramwajowy,365
2,railway_halt,Railway_halt,123
3,railway_station,stacja kolejowa,91
4,taxi,Taxi,79
5,bus_station,Dworzec autobusowy,30
6,ferry_terminal,terminal_promowy,21
7,helipad,lądowisko dla helikopterów,6
8,airport,Lotnisko,1


In [108]:
label_counts.to_markdown()

'|    | fclass          | fclass_PL                  |   value counts |\n|---:|:----------------|:---------------------------|---------------:|\n|  0 | bus_stop        | przystanek autobusowy      |           9391 |\n|  1 | tram_stop       | przystanek tramwajowy      |            365 |\n|  2 | railway_halt    | Railway_halt               |            123 |\n|  3 | railway_station | stacja kolejowa            |             91 |\n|  4 | taxi            | Taxi                       |             79 |\n|  5 | bus_station     | Dworzec autobusowy         |             30 |\n|  6 | ferry_terminal  | terminal_promowy           |             21 |\n|  7 | helipad         | lądowisko dla helikopterów |              6 |\n|  8 | airport         | Lotnisko                   |              1 |'

#water_a

In [109]:
%time
water_a = GeoDataFrame.from_file('maps/gis_osm_water_a_free_1.shp', encoding='utf-8')
water_a["fclass_PL"] = water_a["fclass"].replace(fclass_eng_to_pl)
print(water_a.shape)
water_a.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs
(8792, 6)


,osm_id,code,fclass,name,geometry,fclass_PL
0,5909396,8201,reservoir,Jezioro Klimkowskie,"POLYGON ((21.07813 49.55540, 21.07818 49.55546...",zbiornik
1,5909432,8202,river,Dunajec,"POLYGON ((20.66969 49.60742, 20.67004 49.60778...",rzeka
2,7628519,8200,water,None,"POLYGON ((20.64521 49.79566, 20.64539 49.79588...",woda
3,19796883,8200,water,None,"POLYGON ((19.81997 50.08815, 19.82002 50.08827...",woda
4,19796886,8200,water,None,"POLYGON ((19.82254 50.09034, 19.82400 50.09111...",woda


In [110]:
label_counts = water_a[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,water,woda,6712
1,wetland,teren podmokły,981
2,reservoir,zbiornik,734
3,riverbank,Brzeg rzeki,231
4,river,rzeka,120
5,glacier,lodowiec,12
6,dock,dok,2


In [111]:
label_counts.to_markdown()

'|    | fclass    | fclass_PL      |   value counts |\n|---:|:----------|:---------------|---------------:|\n|  0 | water     | woda           |           6712 |\n|  1 | wetland   | teren podmokły |            981 |\n|  2 | reservoir | zbiornik       |            734 |\n|  3 | riverbank | Brzeg rzeki    |            231 |\n|  4 | river     | rzeka          |            120 |\n|  5 | glacier   | lodowiec       |             12 |\n|  6 | dock      | dok            |              2 |'

#waterways

In [112]:
%time
waterways = GeoDataFrame.from_file('maps/gis_osm_waterways_free_1.shp', encoding='utf-8')
waterways["fclass_PL"] = waterways["fclass"].replace(fclass_eng_to_pl)
print(waterways.shape)
waterways.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs
(46602, 7)


,osm_id,code,fclass,width,name,geometry,fclass_PL
0,5819083,8101,river,0,Biała,"LINESTRING (21.06901 49.44025, 21.06736 49.440...",rzeka
1,5819592,8101,river,0,Dunajec,"LINESTRING (20.68030 49.80995, 20.68049 49.810...",rzeka
2,5909314,8101,river,0,Ropa,"LINESTRING (21.04776 49.60824, 21.04990 49.609...",rzeka
3,5909428,8101,river,0,Dunajec,"LINESTRING (20.65635 49.69135, 20.66124 49.693...",rzeka
4,5909630,8101,river,0,Dunajec,"LINESTRING (20.47315 49.53902, 20.47866 49.537...",rzeka


In [113]:
label_counts = waterways[["fclass","fclass_PL"]].value_counts().to_frame().reset_index().rename(columns={0:"value counts"})
label_counts

,fclass,fclass_PL,value counts
0,stream,strumień,43523
1,river,rzeka,1582
2,drain,drenaż,1300
3,canal,kanał,197


In [114]:
label_counts.to_markdown()

'|    | fclass   | fclass_PL   |   value counts |\n|---:|:---------|:------------|---------------:|\n|  0 | stream   | strumień    |          43523 |\n|  1 | river    | rzeka       |           1582 |\n|  2 | drain    | drenaż      |           1300 |\n|  3 | canal    | kanał       |            197 |'

#maps

In [115]:
# m = folium.Map([50.05,19.95], zoom_start=13,
#                tiles="cartodbpositron"
#                )

# folium.GeoJson(water_a).add_to(m)

# m

In [116]:
# m = folium.Map([50.05,19.95], zoom_start=13,
#                tiles="cartodbpositron"
#                )

# folium.GeoJson(natural).add_to(m)

# m